# Module 03: Memory Management in Distributed Systems

**Difficulty**: ⭐⭐⭐  
**Estimated Time**: 90 minutes  
**Prerequisites**: 
- [Module 00: Setup and Introduction](00_setup_introduction.ipynb)
- [Module 01: Introduction to Distributed Systems](01_introduction_distributed_systems.ipynb)
- [Module 02: Transparency and Architectures](02_transparency_architectures.ipynb)

## Learning Objectives

By the end of this notebook, you will be able to:
1. **Understand** centralized memory management concepts (virtual memory, paging, segmentation)
2. **Differentiate** between Simple, Shared, and Distributed Shared Memory models
3. **Implement** basic shared memory programs using Python's multiprocessing
4. **Analyze** memory migration strategies (replication vs migration)
5. **Compare** OpenMP and MPI programming models
6. **Design** parallel algorithms with appropriate memory management strategies

---

## Section 1: Centralized Memory Management Review

### 1.1 Virtual Memory Fundamentals

Virtual memory extends available memory beyond physical RAM by using disk storage. This is crucial for distributed systems as tasks grow larger.

**Key Concepts:**
- **Virtual Memory**: Extends physical memory using disk storage
- **Paging**: Fixed-size memory blocks
- **Segmentation**: Variable-size memory blocks based on logical divisions
- **Fragmentation**: Wasted memory space
  - **Internal**: Waste within allocated blocks
  - **External**: Waste between allocated blocks

Let's simulate virtual memory with paging:

In [ ]:
# Setup: Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import time
from multiprocessing import Process, Manager, Lock, Queue, cpu_count
import psutil
import os

# Configuration
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)

print(f"✓ Python environment ready")
print(f"✓ Available CPU cores: {cpu_count()}")
print(f"✓ Total system memory: {psutil.virtual_memory().total / (1024**3):.2f} GB")

In [ ]:
class VirtualMemorySimulator:
    """
    Simulates virtual memory with paging.
    
    Parameters:
    -----------
    physical_pages : int
        Number of physical memory pages (RAM)
    page_size : int
        Size of each page in bytes
    """
    
    def __init__(self, physical_pages=4, page_size=1024):
        self.physical_pages = physical_pages
        self.page_size = page_size
        # Page table: maps virtual page number to physical page number
        self.page_table = {}
        # Physical memory (RAM)
        self.physical_memory = [None] * physical_pages
        # Disk storage (unlimited)
        self.disk = {}
        # Track page faults
        self.page_faults = 0
        # LRU tracking
        self.access_order = []
        
    def access_page(self, virtual_page_num):
        """
        Access a virtual page. If not in physical memory, trigger page fault.
        """
        # Check if page is in physical memory
        if virtual_page_num in self.page_table:
            # Page hit - update LRU
            physical_page = self.page_table[virtual_page_num]
            if virtual_page_num in self.access_order:
                self.access_order.remove(virtual_page_num)
            self.access_order.append(virtual_page_num)
            return f"Page {virtual_page_num} HIT (in physical page {physical_page})"
        else:
            # Page fault - need to load from disk
            self.page_faults += 1
            return self._handle_page_fault(virtual_page_num)
    
    def _handle_page_fault(self, virtual_page_num):
        """
        Handle page fault using LRU (Least Recently Used) replacement.
        """
        # Find free physical page
        free_page = None
        for i, page in enumerate(self.physical_memory):
            if page is None:
                free_page = i
                break
        
        if free_page is not None:
            # Load page into free slot
            self.physical_memory[free_page] = virtual_page_num
            self.page_table[virtual_page_num] = free_page
            self.access_order.append(virtual_page_num)
            return f"Page {virtual_page_num} FAULT → Loaded into physical page {free_page}"
        else:
            # Need to evict LRU page
            lru_page = self.access_order.pop(0)
            physical_page = self.page_table[lru_page]
            
            # Evict to disk
            self.disk[lru_page] = self.physical_memory[physical_page]
            del self.page_table[lru_page]
            
            # Load new page
            self.physical_memory[physical_page] = virtual_page_num
            self.page_table[virtual_page_num] = physical_page
            self.access_order.append(virtual_page_num)
            
            return f"Page {virtual_page_num} FAULT → Evicted page {lru_page}, loaded into physical page {physical_page}"
    
    def get_stats(self):
        """Return current statistics."""
        return {
            'page_faults': self.page_faults,
            'pages_in_memory': len(self.page_table),
            'pages_on_disk': len(self.disk)
        }

# Demonstrate virtual memory
vm = VirtualMemorySimulator(physical_pages=4, page_size=1024)

print("Virtual Memory Simulation (4 physical pages)\n")
print("=" * 70)

# Simulate page accesses
access_sequence = [0, 1, 2, 3, 0, 4, 1, 5, 0, 1]

for page_num in access_sequence:
    result = vm.access_page(page_num)
    print(f"Access page {page_num}: {result}")

print("\n" + "=" * 70)
stats = vm.get_stats()
print(f"\nFinal Statistics:")
print(f"  Total Page Faults: {stats['page_faults']}")
print(f"  Pages in Physical Memory: {stats['pages_in_memory']}")
print(f"  Pages on Disk: {stats['pages_on_disk']}")
print(f"  Page Fault Rate: {stats['page_faults']/len(access_sequence)*100:.1f}%")

### 1.2 Fragmentation Demonstration

Fragmentation occurs when memory is allocated and deallocated, leaving unusable gaps.

In [ ]:
def visualize_fragmentation():
    """
    Visualize internal vs external fragmentation.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
    
    # Internal Fragmentation (Fixed-size blocks)
    block_size = 10
    allocations = [7, 9, 5, 8]  # Actual data sizes
    colors_internal = ['skyblue', 'lightcoral', 'lightgreen', 'gold']
    
    x_pos = 0
    for i, alloc in enumerate(allocations):
        # Used space
        ax1.barh(0, alloc, left=x_pos, height=0.8, 
                color=colors_internal[i], edgecolor='black', label=f'Process {i+1}')
        # Wasted space (internal fragmentation)
        waste = block_size - alloc
        ax1.barh(0, waste, left=x_pos+alloc, height=0.8, 
                color='gray', alpha=0.3, edgecolor='black', hatch='//')
        ax1.text(x_pos + block_size/2, 0, f'{waste}B\nwaste', 
                ha='center', va='center', fontsize=9)
        x_pos += block_size
    
    ax1.set_ylim(-0.5, 0.5)
    ax1.set_xlim(0, x_pos)
    ax1.set_xlabel('Memory Address (bytes)')
    ax1.set_title('Internal Fragmentation\n(Fixed 10B blocks, wasted space within blocks)', 
                 fontsize=11, fontweight='bold')
    ax1.set_yticks([])
    ax1.grid(axis='x', alpha=0.3)
    
    # External Fragmentation (Variable-size blocks with gaps)
    allocations_ext = [(0, 15, 'Process A'), (20, 35, 'Process B'), 
                       (40, 52, 'Process C'), (60, 70, 'Process D')]
    colors_ext = ['skyblue', 'lightcoral', 'lightgreen', 'gold']
    
    for i, (start, end, label) in enumerate(allocations_ext):
        ax2.barh(0, end-start, left=start, height=0.8, 
                color=colors_ext[i], edgecolor='black', label=label)
        ax2.text(start + (end-start)/2, 0, label, 
                ha='center', va='center', fontsize=9)
    
    # Mark gaps (external fragmentation)
    gaps = [(15, 20), (35, 40), (52, 60)]
    for start, end in gaps:
        ax2.barh(0, end-start, left=start, height=0.8, 
                color='red', alpha=0.3, edgecolor='black', hatch='xx')
        ax2.text(start + (end-start)/2, 0, f'{end-start}B\nwaste', 
                ha='center', va='center', fontsize=8, color='darkred')
    
    ax2.set_ylim(-0.5, 0.5)
    ax2.set_xlim(0, 75)
    ax2.set_xlabel('Memory Address (bytes)')
    ax2.set_title('External Fragmentation\n(Gaps between variable-size allocations)', 
                 fontsize=11, fontweight='bold')
    ax2.set_yticks([])
    ax2.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate statistics
    total_internal_waste = sum(10 - alloc for alloc in allocations)
    total_external_waste = sum(end - start for start, end in gaps)
    
    print(f"\nFragmentation Analysis:")
    print(f"  Internal Fragmentation: {total_internal_waste}B wasted ({total_internal_waste/(len(allocations)*10)*100:.1f}%)")
    print(f"  External Fragmentation: {total_external_waste}B wasted (gaps between allocations)")

visualize_fragmentation()

---

## Section 2: Shared Memory Models

### 2.1 Simple Memory Model

In the **Simple Memory Model**, all processors have equal access times to memory. This is ideal but rarely practical for large systems.

**Characteristics:**
- Equal memory access times for all processors
- Strict control of multiprogramming degree
- Often avoids virtual memory/caching due to overhead

### 2.2 Shared Memory Model

**Shared Memory** allows multiple processes to share memory locations for inter-process communication (IPC).

**Key Features:**
- Inter-process communication through shared memory
- Memory access via common bus → **bus contention** possible
- Example: OpenMP (directive-based parallelism)
- Threading is the most popular programming technique

Let's implement shared memory communication:

In [ ]:
def worker_process(shared_array, process_id, lock, iterations=1000):
    """
    Worker process that increments shared counter.
    Demonstrates need for synchronization.
    
    Parameters:
    -----------
    shared_array : multiprocessing.Array
        Shared memory array
    process_id : int
        Unique ID for this process
    lock : multiprocessing.Lock
        Lock for synchronization
    iterations : int
        Number of increments to perform
    """
    for _ in range(iterations):
        # Critical section - must be protected
        with lock:
            shared_array[0] += 1
            shared_array[process_id + 1] += 1

def demonstrate_shared_memory():
    """
    Demonstrate shared memory communication between processes.
    """
    from multiprocessing import Array, Lock, Process
    import ctypes
    
    num_processes = 4
    iterations = 5000
    
    # Create shared memory array
    # Index 0: global counter
    # Index 1-4: per-process counters
    shared_array = Array(ctypes.c_int, num_processes + 1)
    lock = Lock()
    
    print(f"Shared Memory Demo: {num_processes} processes, {iterations} iterations each\n")
    print("=" * 70)
    
    # Create and start processes
    processes = []
    start_time = time.time()
    
    for i in range(num_processes):
        p = Process(target=worker_process, 
                   args=(shared_array, i, lock, iterations))
        processes.append(p)
        p.start()
    
    # Wait for all processes to complete
    for p in processes:
        p.join()
    
    end_time = time.time()
    
    # Display results
    print(f"\nResults after {end_time - start_time:.3f} seconds:")
    print(f"  Global Counter: {shared_array[0]} (expected: {num_processes * iterations})")
    print(f"\nPer-Process Counters:")
    for i in range(num_processes):
        print(f"    Process {i}: {shared_array[i+1]} increments")
    
    # Verify correctness
    expected = num_processes * iterations
    actual = shared_array[0]
    if actual == expected:
        print(f"\n✓ SUCCESS: Counter is correct ({actual} == {expected})")
    else:
        print(f"\n✗ ERROR: Counter mismatch ({actual} != {expected})")

demonstrate_shared_memory()

### 2.3 Bus Contention Simulation

**Bus contention** occurs when multiple processors compete for access to shared memory via a common bus.

**Problem**: Systems with >32 CPUs cannot use a single bus efficiently.

In [ ]:
def simulate_bus_contention(num_processors_list=[2, 4, 8, 16, 32, 64]):
    """
    Simulate bus contention as number of processors increases.
    
    Assumption: Bus can handle 1 transaction at a time.
    With more processors, wait time increases.
    """
    # Simulate bus access patterns
    memory_accesses_per_processor = 1000
    bus_bandwidth = 100  # arbitrary units
    
    results = []
    
    for num_procs in num_processors_list:
        # Total requests to bus
        total_requests = num_procs * memory_accesses_per_processor
        
        # With contention, processors must wait
        # Simplified model: each processor gets 1/N of bus bandwidth
        effective_bandwidth_per_proc = bus_bandwidth / num_procs
        
        # Time to complete (inversely proportional to bandwidth)
        completion_time = total_requests / bus_bandwidth
        
        # Contention overhead (waiting time)
        contention_overhead = (num_procs - 1) / num_procs * 100  # percentage
        
        results.append({
            'processors': num_procs,
            'effective_bandwidth': effective_bandwidth_per_proc,
            'contention_overhead': contention_overhead,
            'completion_time': completion_time
        })
    
    # Visualize
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    processors = [r['processors'] for r in results]
    bandwidths = [r['effective_bandwidth'] for r in results]
    overheads = [r['contention_overhead'] for r in results]
    
    # Plot 1: Effective Bandwidth per Processor
    ax1.plot(processors, bandwidths, marker='o', linewidth=2, markersize=8, color='steelblue')
    ax1.set_xlabel('Number of Processors', fontsize=11)
    ax1.set_ylabel('Effective Bandwidth per Processor', fontsize=11)
    ax1.set_title('Bus Contention: Bandwidth Degradation', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.axvline(x=32, color='red', linestyle='--', alpha=0.7, label='32 CPU limit')
    ax1.legend()
    
    # Plot 2: Contention Overhead
    ax2.bar(range(len(processors)), overheads, color='coral', edgecolor='black', alpha=0.7)
    ax2.set_xticks(range(len(processors)))
    ax2.set_xticklabels(processors)
    ax2.set_xlabel('Number of Processors', fontsize=11)
    ax2.set_ylabel('Contention Overhead (%)', fontsize=11)
    ax2.set_title('Bus Contention Overhead', fontsize=12, fontweight='bold')
    ax2.grid(axis='y', alpha=0.3)
    ax2.axhline(y=90, color='red', linestyle='--', alpha=0.7, label='Critical threshold')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    print("\nBus Contention Analysis:")
    print("=" * 70)
    for r in results:
        print(f"{r['processors']:3d} processors: "
              f"Bandwidth/proc = {r['effective_bandwidth']:6.2f}, "
              f"Contention = {r['contention_overhead']:5.1f}%")
    
    print("\n⚠️  Systems with >32 CPUs need alternative interconnects (not single bus)")

simulate_bus_contention()

---

## Section 3: Distributed Shared Memory (DSM)

**Distributed Shared Memory (DSM)** was introduced in 1989 to provide a logical shared memory abstraction for multi-computer systems.

### 3.1 DSM Characteristics

- Presents **logical shared memory** for physically distributed systems
- Maintains communication and data consistency
- Portions of local memory mapped onto DSM
- Directory service tracks all data locations

### 3.2 DSM Management Decisions

1. **How to distribute shared data?**
   - **Replication**: Multiple copies at various locations
   - **Migration**: Single copy moves to different locations

2. **How many readers/writers allowed?**
   - Single writer / multiple readers
   - Multiple writers (requires sophisticated consistency protocols)

In [ ]:
class DistributedSharedMemory:
    """
    Simplified DSM implementation with replication.
    
    Demonstrates basic DSM concepts:
    - Data replication across nodes
    - Consistency maintenance
    - Directory service
    """
    
    def __init__(self, num_nodes=4):
        self.num_nodes = num_nodes
        # Each node has local memory
        self.local_memories = [{} for _ in range(num_nodes)]
        # Directory: tracks which nodes have copies of each data item
        self.directory = {}
        # Statistics
        self.reads = 0
        self.writes = 0
        self.invalidations = 0
        
    def write(self, node_id, key, value):
        """
        Write to DSM. Invalidates all other copies (write-invalidate protocol).
        """
        self.writes += 1
        
        # Write to local memory
        self.local_memories[node_id][key] = value
        
        # Invalidate copies on other nodes
        if key in self.directory:
            for other_node in self.directory[key]:
                if other_node != node_id:
                    if key in self.local_memories[other_node]:
                        del self.local_memories[other_node][key]
                        self.invalidations += 1
        
        # Update directory
        self.directory[key] = {node_id}
        
        return f"Node {node_id} wrote '{key}' = {value} (invalidated {len(self.directory.get(key, set())) - 1} copies)"
    
    def read(self, node_id, key):
        """
        Read from DSM. If not in local memory, fetch from another node.
        """
        self.reads += 1
        
        # Check local memory first
        if key in self.local_memories[node_id]:
            return f"Node {node_id} read '{key}' = {self.local_memories[node_id][key]} (local)"
        
        # Not local - fetch from another node
        if key in self.directory and self.directory[key]:
            source_node = list(self.directory[key])[0]
            value = self.local_memories[source_node][key]
            
            # Replicate to local memory
            self.local_memories[node_id][key] = value
            self.directory[key].add(node_id)
            
            return f"Node {node_id} read '{key}' = {value} (fetched from Node {source_node})"
        
        return f"Node {node_id} read '{key}' = NOT FOUND"
    
    def get_stats(self):
        """Return DSM statistics."""
        return {
            'total_reads': self.reads,
            'total_writes': self.writes,
            'total_invalidations': self.invalidations,
            'unique_keys': len(self.directory)
        }
    
    def visualize_state(self):
        """Visualize current DSM state."""
        print("\nDSM State:")
        print("=" * 70)
        for i, memory in enumerate(self.local_memories):
            print(f"Node {i}: {memory if memory else '(empty)'}")
        print("\nDirectory:")
        for key, nodes in self.directory.items():
            print(f"  '{key}': replicated on nodes {sorted(nodes)}")

# Demonstrate DSM
dsm = DistributedSharedMemory(num_nodes=4)

print("Distributed Shared Memory Simulation\n")
print("=" * 70)

# Sequence of operations
print(dsm.write(0, 'x', 100))
print(dsm.read(1, 'x'))  # Node 1 reads x (will replicate)
print(dsm.read(2, 'x'))  # Node 2 reads x (will replicate)
print(dsm.write(1, 'x', 200))  # Node 1 writes x (invalidates Node 0, 2)
print(dsm.read(0, 'x'))  # Node 0 reads x (must fetch new value)
print(dsm.write(3, 'y', 500))  # Node 3 writes new variable y
print(dsm.read(2, 'y'))  # Node 2 reads y

dsm.visualize_state()

stats = dsm.get_stats()
print("\n" + "=" * 70)
print("\nStatistics:")
print(f"  Total Reads: {stats['total_reads']}")
print(f"  Total Writes: {stats['total_writes']}")
print(f"  Invalidations: {stats['total_invalidations']}")
print(f"  Unique Variables: {stats['unique_keys']}")

### 3.3 DSM Performance Issues

**Key Performance Considerations:**

1. **Data Location**: Where to place original data copy?
2. **Implementation**: Hardware vs software vs hybrid?
3. **Thrashing**: Avoid constant data movement between locations
4. **Block Size**: Optimal size for sharable data blocks (page size, packet size, segment size)?

**Non-local memory references are expensive** (up to 10:1 ratio compared to local access).

In [ ]:
def simulate_memory_access_latency():
    """
    Demonstrate local vs remote memory access latency.
    """
    # Latency values (nanoseconds)
    access_types = [
        ('L1 Cache', 1),
        ('L2 Cache', 4),
        ('L3 Cache', 10),
        ('Local RAM', 100),
        ('Remote RAM\n(same node)', 200),
        ('Remote RAM\n(different node)', 1000)
    ]
    
    labels = [name for name, _ in access_types]
    latencies = [latency for _, latency in access_types]
    colors = ['green', 'lightgreen', 'yellow', 'orange', 'coral', 'red']
    
    fig, ax = plt.subplots(figsize=(12, 6))
    bars = ax.barh(labels, latencies, color=colors, edgecolor='black', alpha=0.7)
    
    # Add value labels
    for i, (bar, latency) in enumerate(zip(bars, latencies)):
        ax.text(latency + 20, i, f'{latency} ns', 
               va='center', fontsize=10, fontweight='bold')
    
    ax.set_xlabel('Latency (nanoseconds)', fontsize=12)
    ax.set_title('Memory Access Latency Hierarchy\n(Remote access can be 10x slower)', 
                fontsize=13, fontweight='bold')
    ax.set_xscale('log')
    ax.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\nMemory Access Latency Comparison:")
    print("=" * 70)
    base_latency = latencies[3]  # Local RAM as baseline
    for name, latency in access_types:
        ratio = latency / base_latency
        print(f"{name:25s}: {latency:5d} ns ({ratio:4.1f}x local RAM)")

simulate_memory_access_latency()

---

## Section 4: Memory Migration

### 4.1 Replication vs Migration

Two fundamental strategies for distributing data in DSM:

| **Replication** | **Migration** |
|-----------------|---------------|
| Multiple copies at various locations | Single copy moves between locations |
| Faster reads (local copy) | Simpler consistency (only one copy) |
| Complex write consistency | Slower if frequently accessed from multiple nodes |
| More storage overhead | Less storage overhead |

### 4.2 Migration Decisions

**Two fundamental questions:**
1. **When** in the migration process to migrate memory?
2. **How much** memory needs to be migrated?

In [ ]:
class MemoryMigrationSimulator:
    """
    Simulates memory migration vs replication strategies.
    """
    
    def __init__(self, num_nodes=4):
        self.num_nodes = num_nodes
        self.migration_data = {}  # key -> node_id
        self.replication_data = {}  # key -> set of node_ids
        
        # Costs
        self.migration_moves = 0
        self.replication_copies = 0
        self.replication_invalidations = 0
        
    def migration_access(self, node_id, key, is_write=False):
        """
        Access data using migration strategy.
        Data moves to the accessing node.
        """
        if key not in self.migration_data:
            # First write
            self.migration_data[key] = node_id
            return f"Migration: Node {node_id} created '{key}'"
        
        current_node = self.migration_data[key]
        
        if current_node != node_id:
            # Need to migrate
            self.migration_data[key] = node_id
            self.migration_moves += 1
            return f"Migration: Moved '{key}' from Node {current_node} to Node {node_id}"
        else:
            return f"Migration: Node {node_id} accessed '{key}' (local)"
    
    def replication_access(self, node_id, key, is_write=False):
        """
        Access data using replication strategy.
        Reads create copies; writes invalidate other copies.
        """
        if key not in self.replication_data:
            # First write
            self.replication_data[key] = {node_id}
            return f"Replication: Node {node_id} created '{key}'"
        
        if is_write:
            # Write: invalidate all other copies
            num_invalidated = len(self.replication_data[key]) - (1 if node_id in self.replication_data[key] else 0)
            self.replication_invalidations += num_invalidated
            self.replication_data[key] = {node_id}
            return f"Replication: Node {node_id} wrote '{key}' (invalidated {num_invalidated} copies)"
        else:
            # Read: create copy if not present
            if node_id not in self.replication_data[key]:
                self.replication_data[key].add(node_id)
                self.replication_copies += 1
                return f"Replication: Node {node_id} read '{key}' (created local copy)"
            else:
                return f"Replication: Node {node_id} read '{key}' (local copy)"
    
    def compare_strategies(self, access_sequence):
        """
        Compare migration vs replication for a given access sequence.
        
        access_sequence: list of tuples (node_id, key, is_write)
        """
        print("\nComparing Migration vs Replication Strategies\n")
        print("=" * 70)
        print(f"Access Sequence: {len(access_sequence)} operations\n")
        
        # Reset statistics
        self.migration_moves = 0
        self.replication_copies = 0
        self.replication_invalidations = 0
        
        # Process accesses
        for node_id, key, is_write in access_sequence:
            self.migration_access(node_id, key, is_write)
            self.replication_access(node_id, key, is_write)
        
        # Display results
        print("Migration Strategy:")
        print(f"  Data movements: {self.migration_moves}")
        print(f"  Storage overhead: 1x (single copy)")
        print(f"  Total cost: {self.migration_moves} moves\n")
        
        print("Replication Strategy:")
        print(f"  Copies created: {self.replication_copies}")
        print(f"  Invalidations: {self.replication_invalidations}")
        total_copies = sum(len(nodes) for nodes in self.replication_data.values())
        print(f"  Storage overhead: {total_copies}x (multiple copies)")
        print(f"  Total cost: {self.replication_copies + self.replication_invalidations} operations\n")
        
        # Visualize
        fig, ax = plt.subplots(figsize=(10, 5))
        
        strategies = ['Migration', 'Replication']
        costs = [
            self.migration_moves,
            self.replication_copies + self.replication_invalidations
        ]
        colors_bar = ['skyblue', 'coral']
        
        bars = ax.bar(strategies, costs, color=colors_bar, edgecolor='black', alpha=0.7)
        
        # Add value labels
        for bar, cost in zip(bars, costs):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{int(cost)} ops',
                   ha='center', va='bottom', fontsize=12, fontweight='bold')
        
        ax.set_ylabel('Total Operations (cost)', fontsize=12)
        ax.set_title('Migration vs Replication: Performance Comparison', 
                    fontsize=13, fontweight='bold')
        ax.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Recommendation
        if self.migration_moves < (self.replication_copies + self.replication_invalidations):
            print("\n✓ Recommendation: MIGRATION is more efficient for this access pattern")
        else:
            print("\n✓ Recommendation: REPLICATION is more efficient for this access pattern")

# Test with different access patterns
sim = MemoryMigrationSimulator(num_nodes=4)

# Access pattern: Mostly reads from different nodes (favors replication)
access_pattern_1 = [
    (0, 'data', True),   # Node 0 writes
    (1, 'data', False),  # Node 1 reads
    (2, 'data', False),  # Node 2 reads
    (3, 'data', False),  # Node 3 reads
    (0, 'data', False),  # Node 0 reads
    (1, 'data', False),  # Node 1 reads
]

print("\n📊 Test Case 1: Read-heavy workload (multiple nodes reading)")
sim.compare_strategies(access_pattern_1)

In [ ]:
# Access pattern: Alternating writes from different nodes (favors migration)
access_pattern_2 = [
    (0, 'data', True),   # Node 0 writes
    (0, 'data', False),  # Node 0 reads
    (0, 'data', False),  # Node 0 reads
    (1, 'data', True),   # Node 1 writes (invalidates)
    (1, 'data', False),  # Node 1 reads
    (2, 'data', True),   # Node 2 writes (invalidates)
    (2, 'data', False),  # Node 2 reads
]

print("\n📊 Test Case 2: Write-heavy workload (alternating writers)")
sim2 = MemoryMigrationSimulator(num_nodes=4)
sim2.compare_strategies(access_pattern_2)

---

## Section 5: Programming Models Comparison

### 5.1 OpenMP vs MPI

| **OpenMP** | **MPI** |
|------------|----------|
| **Shared Memory** model | **Distributed Memory** model |
| Add pragmas to existing code | Must rewrite for message passing |
| Compiler handles parallelization | Explicit data movement |
| Easy to use | More control, more complex |
| Limited to single node | Works across nodes |
| Threading-based | Process-based |

### 5.2 Synchronous vs Asynchronous Communication

**Synchronous**:
- `send()` returns when receiver acknowledges
- `recv()` returns when data copied to receiver
- Potential for deadlock

**Asynchronous**:
- `send()` returns immediately
- `recv()` posts intent, returns immediately  
- Use `checksend()`, `checkrecv()` to verify status
- More flexible, better performance

Let's simulate message passing:

In [ ]:
def simulate_message_passing():
    """
    Simulate MPI-style message passing between processes.
    """
    from multiprocessing import Process, Queue
    import time
    
    def sender_process(send_queue, process_id, num_messages=5):
        """Send messages to queue."""
        for i in range(num_messages):
            message = f"Message {i} from Process {process_id}"
            send_queue.put(message)
            print(f"  [Sender {process_id}] Sent: {message}")
            time.sleep(0.1)  # Simulate work
        send_queue.put(None)  # Sentinel to indicate done
    
    def receiver_process(recv_queue, process_id, num_senders=2):
        """Receive messages from queue."""
        received = 0
        done_senders = 0
        
        while done_senders < num_senders:
            message = recv_queue.get()
            if message is None:
                done_senders += 1
            else:
                received += 1
                print(f"  [Receiver {process_id}] Received: {message}")
        
        print(f"\n  [Receiver {process_id}] Total messages received: {received}")
    
    print("\nMessage Passing Simulation (MPI-style)\n")
    print("=" * 70)
    
    # Create communication queue
    message_queue = Queue()
    
    # Create sender processes
    num_senders = 2
    senders = []
    for i in range(num_senders):
        p = Process(target=sender_process, args=(message_queue, i, 3))
        senders.append(p)
        p.start()
    
    # Create receiver process
    receiver = Process(target=receiver_process, args=(message_queue, 0, num_senders))
    receiver.start()
    
    # Wait for all to complete
    for p in senders:
        p.join()
    receiver.join()
    
    print("\n" + "=" * 70)
    print("✓ Message passing complete")

simulate_message_passing()

---

## Section 6: Practical Exercises

### Exercise 1: Page Replacement Algorithm

Implement and compare different page replacement algorithms:
- FIFO (First-In-First-Out)
- LRU (Least Recently Used)
- Optimal (theoretical best)

In [ ]:
def fifo_page_replacement(page_sequence, num_frames):
    """
    EXERCISE: Implement FIFO page replacement algorithm.
    
    Parameters:
    -----------
    page_sequence : list
        Sequence of page accesses
    num_frames : int
        Number of available page frames
    
    Returns:
    --------
    int : Number of page faults
    
    TODO: Complete this implementation
    Hint: Use a queue to track insertion order
    """
    frames = []
    page_faults = 0
    
    for page in page_sequence:
        # YOUR CODE HERE
        pass
    
    return page_faults

# Test your implementation
test_sequence = [7, 0, 1, 2, 0, 3, 0, 4, 2, 3, 0, 3, 2]
faults = fifo_page_replacement(test_sequence, num_frames=3)
print(f"FIFO Page Faults: {faults}")
print(f"Expected: ~12 page faults")

### Exercise 2: Shared Memory Producer-Consumer

Implement a producer-consumer pattern using shared memory.

In [ ]:
def producer_consumer_exercise():
    """
    EXERCISE: Implement producer-consumer with shared memory.
    
    Requirements:
    1. Create a shared buffer (queue) of size 5
    2. Producer adds items to buffer
    3. Consumer removes items from buffer
    4. Use locks to prevent race conditions
    5. Handle buffer full/empty conditions
    
    TODO: Complete the producer and consumer functions below
    """
    from multiprocessing import Process, Queue, Lock
    import time
    
    def producer(shared_queue, lock, items_to_produce):
        """
        Produce items and add to shared queue.
        YOUR CODE HERE
        """
        pass
    
    def consumer(shared_queue, lock, items_to_consume):
        """
        Consume items from shared queue.
        YOUR CODE HERE
        """
        pass
    
    # Test your implementation
    buffer = Queue(maxsize=5)
    lock = Lock()
    
    # Create processes
    # YOUR CODE HERE
    
    print("Producer-Consumer exercise complete!")

# Uncomment to test
# producer_consumer_exercise()

### Exercise 3: DSM Consistency Protocol

Implement a simple DSM consistency protocol.

In [ ]:
def dsm_consistency_exercise():
    """
    EXERCISE: Extend the DSM implementation with:
    
    1. Write-update protocol (instead of write-invalidate)
    2. Track number of messages sent
    3. Compare message overhead of update vs invalidate
    
    Write-Update: On write, send new value to all nodes with copies
    Write-Invalidate: On write, invalidate all other copies
    
    TODO: Implement write-update DSM class
    """
    
    class DSM_WriteUpdate:
        """YOUR CODE HERE"""
        pass
    
    # Test both protocols and compare
    print("Compare Write-Update vs Write-Invalidate protocols")
    # YOUR CODE HERE

# Uncomment to test
# dsm_consistency_exercise()

---

## Summary

### Key Concepts Covered

1. **Centralized Memory Management**
   - Virtual memory extends physical memory using disk
   - Paging (fixed-size) vs Segmentation (variable-size)
   - Internal vs External fragmentation
   - Page replacement algorithms (LRU, FIFO)

2. **Memory Models**
   - **Simple Memory**: Equal access times, simple but limited
   - **Shared Memory**: IPC via shared locations, bus contention issues
   - **Distributed Shared Memory**: Logical shared memory across physical nodes

3. **DSM Management**
   - **Replication**: Multiple copies, fast reads, complex consistency
   - **Migration**: Single copy moves, simpler consistency, slower multi-node access
   - Directory service tracks data locations

4. **Programming Models**
   - **OpenMP**: Shared memory, pragma-based, easy
   - **MPI**: Distributed memory, explicit messages, scalable
   - Synchronous vs Asynchronous communication

5. **Performance Considerations**
   - Bus contention limits scalability (>32 CPUs)
   - Non-local memory access is 10x slower
   - Trade-offs between consistency overhead and performance

### What's Next?

In the next module, we'll explore:
- Real-time systems and their requirements
- Clock synchronization in distributed systems
- Distributed operating systems

---

## Additional Resources

### Documentation
- [OpenMP Official Documentation](https://www.openmp.org/)
- [MPI Tutorial](https://mpitutorial.com/)
- [Python Multiprocessing](https://docs.python.org/3/library/multiprocessing.html)

### Further Reading
- Tanenbaum & Van Steen: *Distributed Systems: Principles and Paradigms*
- Hennessy & Patterson: *Computer Architecture: A Quantitative Approach*

### Practice
- [Lawrence Livermore OpenMP Tutorial](https://hpc-tutorials.llnl.gov/openmp/)
- [MPI Examples](https://www.mcs.anl.gov/research/projects/mpi/tutorial/)

---

**Congratulations!** You've completed Module 03: Memory Management in Distributed Systems. 🎉